## Interaktiver Phi 3 Mini 4K Instruct Chatbot mit Whisper

### Einführung:
Der interaktive Phi 3 Mini 4K Instruct Chatbot ist ein Tool, das es Nutzern ermöglicht, mit der Microsoft Phi 3 Mini 4K Instruct-Demo über Text- oder Audioeingaben zu interagieren. Der Chatbot kann für eine Vielzahl von Aufgaben verwendet werden, wie Übersetzungen, Wetteraktualisierungen und allgemeine Informationsbeschaffung.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Erstellen Sie Ihren Huggingface Access Token

Erstellen Sie einen neuen Token  
Geben Sie einen neuen Namen an  
Wählen Sie Schreibberechtigungen aus  
Kopieren Sie den Token und speichern Sie ihn an einem sicheren Ort


Die folgende Python-Datei führt zwei Hauptaufgaben aus: das Importieren des `os`-Moduls und das Setzen einer Umgebungsvariable.

1. Importieren des `os`-Moduls:
   - Das `os`-Modul in Python bietet eine Möglichkeit, mit dem Betriebssystem zu interagieren. Es ermöglicht verschiedene betriebssystembezogene Aufgaben, wie den Zugriff auf Umgebungsvariablen, das Arbeiten mit Dateien und Verzeichnissen usw.
   - In diesem Code wird das `os`-Modul mit der `import`-Anweisung importiert. Diese Anweisung macht die Funktionalität des `os`-Moduls für die Verwendung im aktuellen Python-Skript verfügbar.

2. Setzen einer Umgebungsvariable:
   - Eine Umgebungsvariable ist ein Wert, der von Programmen, die auf dem Betriebssystem laufen, abgerufen werden kann. Sie dient dazu, Konfigurationseinstellungen oder andere Informationen zu speichern, die von mehreren Programmen verwendet werden können.
   - In diesem Code wird eine neue Umgebungsvariable mithilfe des `os.environ`-Dictionary gesetzt. Der Schlüssel des Dictionarys ist `'HF_TOKEN'`, und der Wert wird aus der `HUGGINGFACE_TOKEN`-Variable zugewiesen.
   - Die `HUGGINGFACE_TOKEN`-Variable wird direkt oberhalb dieses Code-Snippets definiert und erhält einen Zeichenkettenwert `"hf_**************"` mithilfe der `#@param`-Syntax. Diese Syntax wird häufig in Jupyter-Notebooks verwendet, um Benutzereingaben und Parameterkonfiguration direkt in der Notebook-Oberfläche zu ermöglichen.
   - Durch das Setzen der Umgebungsvariable `'HF_TOKEN'` kann sie von anderen Teilen des Programms oder von anderen Programmen, die auf demselben Betriebssystem laufen, abgerufen werden.

Zusammengefasst importiert dieser Code das `os`-Modul und setzt eine Umgebungsvariable namens `'HF_TOKEN'` mit dem Wert, der in der `HUGGINGFACE_TOKEN`-Variable bereitgestellt wird.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Dieser Codeausschnitt definiert eine Funktion namens clear_output, die verwendet wird, um die Ausgabe der aktuellen Zelle in Jupyter Notebook oder IPython zu löschen. Lassen Sie uns den Code aufschlüsseln und seine Funktionalität verstehen:

Die Funktion clear_output nimmt einen Parameter namens wait, der einen booleschen Wert darstellt. Standardmäßig ist wait auf False gesetzt. Dieser Parameter bestimmt, ob die Funktion warten soll, bis neue Ausgabe verfügbar ist, um die bestehende Ausgabe zu ersetzen, bevor sie gelöscht wird.

Die Funktion selbst wird verwendet, um die Ausgabe der aktuellen Zelle zu löschen. In Jupyter Notebook oder IPython wird die Ausgabe einer Zelle, wie gedruckter Text oder grafische Darstellungen, unterhalb der Zelle angezeigt. Die Funktion clear_output ermöglicht es, diese Ausgabe zu entfernen.

Die Implementierung der Funktion ist im Codeausschnitt nicht enthalten, wie durch die Auslassungspunkte (...) angezeigt wird. Die Auslassungspunkte stellen einen Platzhalter für den tatsächlichen Code dar, der das Löschen der Ausgabe durchführt. Die Implementierung der Funktion könnte die Interaktion mit der Jupyter Notebook- oder IPython-API beinhalten, um die bestehende Ausgabe aus der Zelle zu entfernen.

Insgesamt bietet diese Funktion eine praktische Möglichkeit, die Ausgabe der aktuellen Zelle in Jupyter Notebook oder IPython zu löschen, was die Verwaltung und Aktualisierung der angezeigten Ausgabe während interaktiver Codingsitzungen erleichtert.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Führen Sie Text-to-Speech (TTS) mit dem Edge TTS-Dienst aus. Lassen Sie uns die relevanten Funktionsimplementierungen nacheinander durchgehen:

1. `calculate_rate_string(input_value)`: Diese Funktion nimmt einen Eingabewert und berechnet die Geschwindigkeitszeichenkette für die TTS-Stimme. Der Eingabewert repräsentiert die gewünschte Sprechgeschwindigkeit, wobei ein Wert von 1 die normale Geschwindigkeit darstellt. Die Funktion berechnet die Geschwindigkeitszeichenkette, indem sie 1 vom Eingabewert subtrahiert, ihn mit 100 multipliziert und dann das Vorzeichen basierend darauf bestimmt, ob der Eingabewert größer oder gleich 1 ist. Die Funktion gibt die Geschwindigkeitszeichenkette im Format "{Vorzeichen}{Geschwindigkeit}" zurück.

2. `make_chunks(input_text, language)`: Diese Funktion nimmt einen Eingabetext und eine Sprache als Parameter. Sie teilt den Eingabetext basierend auf sprachspezifischen Regeln in Abschnitte. In dieser Implementierung teilt die Funktion den Text, wenn die Sprache "Englisch" ist, an jedem Punkt (".") und entfernt führende oder nachfolgende Leerzeichen. Anschließend fügt sie jedem Abschnitt einen Punkt hinzu und gibt die gefilterte Liste der Abschnitte zurück.

3. `tts_file_name(text)`: Diese Funktion generiert einen Dateinamen für die TTS-Audiodatei basierend auf dem Eingabetext. Sie führt mehrere Transformationen am Text durch: Entfernen eines abschließenden Punktes (falls vorhanden), Umwandeln des Textes in Kleinbuchstaben, Entfernen von führenden und nachfolgenden Leerzeichen und Ersetzen von Leerzeichen durch Unterstriche. Anschließend kürzt sie den Text auf maximal 25 Zeichen (falls länger) oder verwendet den vollständigen Text, wenn er leer ist. Schließlich generiert sie eine zufällige Zeichenfolge mit dem [`uuid`]-Modul und kombiniert sie mit dem gekürzten Text, um den Dateinamen im Format "/content/edge_tts_voice/{gekürzter_text}_{zufällige_zeichenfolge}.mp3" zu erstellen.

4. `merge_audio_files(audio_paths, output_path)`: Diese Funktion kombiniert mehrere Audiodateien zu einer einzigen Audiodatei. Sie nimmt eine Liste von Audiodateipfaden und einen Ausgabepfad als Parameter. Die Funktion initialisiert ein leeres `AudioSegment`-Objekt namens [`merged_audio`]. Anschließend iteriert sie durch jeden Audiodateipfad, lädt die Audiodatei mit der Methode `AudioSegment.from_file()` aus der `pydub`-Bibliothek und fügt die aktuelle Audiodatei dem [`merged_audio`]-Objekt hinzu. Schließlich exportiert sie das zusammengeführte Audio im MP3-Format an den angegebenen Ausgabepfad.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Diese Funktion führt die TTS-Operation mit dem Edge TTS-Dienst aus. Sie nimmt eine Liste von Textabschnitten, die Sprechgeschwindigkeit, den Namen der Stimme und den Speicherpfad als Parameter. Wenn die Anzahl der Abschnitte größer als 1 ist, erstellt die Funktion ein Verzeichnis zum Speichern der einzelnen Abschnitts-Audiodateien. Anschließend iteriert sie durch jeden Abschnitt, erstellt einen Edge TTS-Befehl mit der Funktion `calculate_rate_string()`, dem Namen der Stimme und dem Abschnittstext und führt den Befehl mit der Funktion `os.system()` aus. Wenn die Befehlsausführung erfolgreich ist, fügt sie den Pfad der generierten Audiodatei einer Liste hinzu. Nach der Verarbeitung aller Abschnitte kombiniert sie die einzelnen Audiodateien mit der Funktion `merge_audio_files()` und speichert das zusammengeführte Audio am angegebenen Speicherpfad. Wenn es nur einen Abschnitt gibt, generiert sie direkt den Edge TTS-Befehl und speichert das Audio am Speicherpfad. Schließlich gibt sie den Speicherpfad der generierten Audiodatei zurück.

6. `random_audio_name_generate()`: Diese Funktion generiert einen zufälligen Audiodateinamen mit dem [`uuid`]-Modul. Sie generiert eine zufällige UUID, wandelt sie in eine Zeichenkette um, nimmt die ersten 8 Zeichen, fügt die Erweiterung ".mp3" hinzu und gibt den zufälligen Audiodateinamen zurück.

7. `talk(input_text)`: Diese Funktion ist der Haupteinstiegspunkt für die Durchführung der TTS-Operation. Sie nimmt einen Eingabetext als Parameter. Zunächst überprüft sie die Länge des Eingabetextes, um festzustellen, ob es sich um einen langen Satz handelt (größer oder gleich 600 Zeichen). Basierend auf der Länge und dem Wert der Variablen `translate_text_flag` bestimmt sie die Sprache und generiert die Liste der Textabschnitte mit der Funktion `make_chunks()`. Anschließend generiert sie einen Speicherpfad für die Audiodatei mit der Funktion `random_audio_name_generate()`. Schließlich ruft sie die Funktion `edge_free_tts()` auf, um die TTS-Operation durchzuführen, und gibt den Speicherpfad der generierten Audiodatei zurück.

Insgesamt arbeiten diese Funktionen zusammen, um den Eingabetext in Abschnitte zu unterteilen, einen Dateinamen für die Audiodatei zu generieren, die TTS-Operation mit dem Edge TTS-Dienst auszuführen und die einzelnen Audiodateien zu einer einzigen Audiodatei zusammenzuführen.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Die Implementierung von zwei Funktionen: convert_to_text und run_text_prompt, sowie die Deklaration von zwei Klassen: str und Audio.

Die Funktion convert_to_text nimmt einen audio_path als Eingabe und transkribiert die Audiodatei mithilfe eines Modells namens whisper_model in Text. Zunächst wird überprüft, ob das gpu-Flag auf True gesetzt ist. Falls ja, wird das whisper_model mit bestimmten Parametern wie word_timestamps=True, fp16=True, language='English' und task='translate' verwendet. Wenn das gpu-Flag auf False gesetzt ist, wird das whisper_model mit fp16=False verwendet. Die resultierende Transkription wird anschließend in einer Datei namens 'scan.txt' gespeichert und als Text zurückgegeben.

Die Funktion run_text_prompt nimmt eine Nachricht und eine chat_history als Eingabe. Sie verwendet die Funktion phi_demo, um basierend auf der Eingabenachricht eine Antwort von einem Chatbot zu generieren. Die generierte Antwort wird dann an die Funktion talk übergeben, die die Antwort in eine Audiodatei umwandelt und den Dateipfad zurückgibt. Die Klasse Audio wird verwendet, um die Audiodatei anzuzeigen und abzuspielen. Das Audio wird mit der display-Funktion aus dem Modul IPython.display angezeigt, und das Audio-Objekt wird mit dem Parameter autoplay=True erstellt, sodass die Wiedergabe automatisch startet. Die chat_history wird mit der Eingabenachricht und der generierten Antwort aktualisiert, und ein leerer String sowie die aktualisierte chat_history werden zurückgegeben.

Die Klasse str ist eine eingebaute Klasse in Python, die eine Zeichenfolge repräsentiert. Sie bietet verschiedene Methoden zur Manipulation und Bearbeitung von Zeichenfolgen, wie z. B. capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill und mehr. Diese Methoden ermöglichen Operationen wie Suchen, Ersetzen, Formatieren und Bearbeiten von Zeichenfolgen.

Die Klasse Audio ist eine benutzerdefinierte Klasse, die ein Audio-Objekt repräsentiert. Sie wird verwendet, um einen Audio-Player in der Jupyter Notebook-Umgebung zu erstellen. Die Klasse akzeptiert verschiedene Parameter wie data, filename, url, embed, rate, autoplay und normalize. Der Parameter data kann ein numpy-Array, eine Liste von Samples, ein String, der einen Dateinamen oder eine URL repräsentiert, oder rohe PCM-Daten sein. Der Parameter filename wird verwendet, um eine lokale Datei anzugeben, aus der die Audiodaten geladen werden sollen, und der Parameter url wird verwendet, um eine URL anzugeben, von der die Audiodaten heruntergeladen werden sollen. Der Parameter embed bestimmt, ob die Audiodaten mithilfe eines Data-URI eingebettet oder aus der Originalquelle referenziert werden sollen. Der Parameter rate gibt die Abtastrate der Audiodaten an. Der Parameter autoplay bestimmt, ob die Wiedergabe des Audios automatisch starten soll. Der Parameter normalize gibt an, ob die Audiodaten normalisiert (auf den maximal möglichen Bereich skaliert) werden sollen. Die Klasse Audio bietet außerdem Methoden wie reload, um die Audiodaten aus einer Datei oder URL neu zu laden, sowie Attribute wie src_attr, autoplay_attr und element_id_attr, um die entsprechenden Attribute für das Audio-Element in HTML abzurufen.

Insgesamt werden diese Funktionen und Klassen verwendet, um Audio in Text zu transkribieren, Audioantworten von einem Chatbot zu generieren und Audio in der Jupyter Notebook-Umgebung anzuzeigen und abzuspielen.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
